In [ ]:
import numpy as np

In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    # idxには抽出する行のインデックス（単語ID）を格納
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    #　重みの勾配をdwとして取り出す
    # dW[...]=0でdWの要素を0で上書きする（dWを0にするのではなく、dWの形状を保ったまま、その要素を0にする）
    # そして前走から伝わってくる勾配doutをidxで指定された行に代入する
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        # 以下ではidxが重複したときに、どっちかの値が消えてしまう
        dW[self.idx] = dout
        
        # 上記問題を解決した仕様が下記
        # 勾配を加算していく
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None
    
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.forward(W)
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W*h, axis=1)
        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0] , 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh
    
# Negative Samplingとは
# 正例をターゲットとした場合の損失の算出と同時に、負例をいくつかサンプリングして同様に損失を算出する
# そして。各データの損失を足し合わせて、最終的な損失とする
# サンプリングの方法はコーパスから確率分布を作成し、その確率分布からサンプリングする
# 具体的なフロー
# コーパスから単語の確率分布を作成し、それを0.75乗する。
# これらの処理をUnigramSamplerクラスを使って行う
# corpus：単語IDのリスト、power：確率分布に対する累乗（デフォルトは0.75）、sample_size：負例サンプリングを行う個数
class NegativeSamplingloss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithloss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []
        
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample= self.sampler.get_negative_sample(target)
        
        # 正例のforward
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype = np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 負例のforward
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
    
    def backward(self, dout = 1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        return dh